In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting
from matplotlib.ticker import MaxNLocator

In [ ]:
def pendulum(x, l = 1., g = 9.81):
    theta_ddot = (-g/l)*np.sin(x[:1])

    return np.concatenate([x[1:], theta_ddot], axis=0)

def deriv_pendulum(x, l = 1., g = 9.81):
    dx = np.zeros((2, 2)) 
    dx[0, 1] = 1.
    dx[1:, 0] = (-g/l)*np.cos(x[0])
    return dx

In [ ]:
def backward_euler_fixed_point(x0, dt):
    x = np.copy(x0)
    errors = [np.linalg.norm(x0 + pendulum(x)*dt - x)]

    while errors[-1] > 1e-8:
        x = x0 + pendulum(x) * dt
        errors.append(np.linalg.norm(x0 + pendulum(x) * dt - x))
    
    return x, errors

In [ ]:
def backward_euler_newton(x0, dt):
    x = np.copy(x0)

    r = x0 + pendulum(x)*dt - x
    errors = [np.linalg.norm(r)]

    while errors[-1] > 1e-8:
        grad_r = deriv_pendulum(x) * dt - np.eye(x.shape[0])

        dx = -np.linalg.inv(grad_r) @ r
        x = x + dx
        r = x0 + pendulum(x) * dt - x
        errors.append(np.linalg.norm(r))
    
    return x, errors

In [ ]:
def forward_simulation(int_func, x0, dt, total_time):
    times = [0.]
    states = [np.copy(x0)]

    t = 0.
    x = np.copy(x0)
    while t < (total_time - 1e-6):
        x, _ = int_func(x, dt)
        t += dt
        states.append(np.copy(x))
        times.append(t)
    
    return states, times

In [ ]:
def plot_hist_all(hist1, hist2, t, plot_vel = False):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.plot(t, [h[0] for h in hist1], color='black')
    if plot_vel:
        ax.plot(t, [h[1] for h in hist1], color='yellow', alpha=0.3)
    ax.plot(t, [h[0] for h in hist2], color='orange')
    if plot_vel:
        ax.plot(t, [h[1] for h in hist2], color='gary', alpha=0.3)

    plt.show()

In [ ]:
def plot_errors(e1, e2):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    plt.semilogy(e1, label='Fixed Point Iteration')
    plt.semilogy(e2, label='Newton\'s Method')

    ax.legend()

    ax.set_xlabel('Iterations')
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylabel('Error')

    plt.show()

In [ ]:
# Initial state
x0 = np.array([[0.1, 0.]]).T

dt = 0.01
total_time = 10.

# Backward Euler with Fixed Point Iteration
hist1, t = forward_simulation(backward_euler_fixed_point, x0, dt, total_time)

In [ ]:
# Backward Euler with Newton's Method
hist2, t = forward_simulation(backward_euler_newton, x0, dt, total_time)

In [ ]:
# Let's see the results
plot_hist_all(hist1, hist2, t)

In [ ]:
# Let's print the errors for each timestep
dt = 0.1
x1, e1 = backward_euler_fixed_point(x0, dt)
x2, e2 = backward_euler_newton(x0, dt)
print("Fixed Point Iteration:")
for e in e1:
    print(e)
print("Newton's Method:")
for e in e2:
    print(e)

In [ ]:
# Let's plot them as well
plot_errors(e1, e2)